In [ ]:
import time
import csv
import os
from ansys.mechanical.core import embedding as app
from ansys.mechanical.core import global_variables

In [ ]:
# inputs_dir = r"D:\mech\inputs"
# work_dir = r"D:\mech\wdir"

inputs_dir = os.path.join(os.getcwd(),"inputs")
outputs_dir = os.path.join(os.getcwd(),"outputs")
work_dir = os.path.join(os.getcwd(),"wdir")

In [ ]:
ansys_version = 241

# geometryfilewithpath = os.path.join(inputs_dir, "Valve_Geometry.scdoc")
geometryfilewithpath = os.path.join(inputs_dir, "Valve_Geometry_NS.scdoc")
matfilewithpath = os.path.join(inputs_dir, "MatML2.xml")
mechdat_filepath = os.path.join(work_dir, "embedded_105.mechdb") # edit this filename

In [ ]:

e_app = app.App(version=ansys_version)
print(e_app)
globals().update(global_variables(e_app, True))


In [ ]:

"""Import board geometry with specified settings"""
geometry_import = Model.GeometryImportGroup.AddGeometryImport()
geometry_import_format = (Ansys.Mechanical.DataModel.Enums.GeometryImportPreference.Format.Automatic)
geometry_import_preferences = (Ansys.ACT.Mechanical.Utilities.GeometryImportPreferences())
geometry_import_preferences.NamedSelectionKey = ""
geometry_import_preferences.ProcessNamedSelections = True
geometry_import.Import(geometryfilewithpath, geometry_import_format, geometry_import_preferences)

"""Import Materials from xml file created from Engineering Data"""
Model.Materials.Import(matfilewithpath)
e_app.save(mechdat_filepath)


In [ ]:

allbodies=ExtAPI.DataModel.Project.Model.GetChildren( Ansys.Mechanical.DataModel.Enums.DataModelObjectCategory.Body,True)
print("\nTotal Number of Bodies : ", allbodies.Count)

print("Materials available : ")
for mat in Model.Materials.Children:
    print(mat.Name)



In [ ]:
part =  ExtAPI.DataModel.GetObjectsByName("Connector")[0]
part.Material = "Gray Cast Iron"
ExtAPI.DataModel.Tree.Refresh()

e_app.save(mechdat_filepath)

In [ ]:
# Define mesh settings
mesh = Model.Mesh
mesh.ElementSize = Quantity('25 [mm]')
mesh.GenerateMesh()



In [ ]:

# Define boundary conditions:
analysis = Model.AddStaticStructuralAnalysis()

fixedSupport = analysis.AddFixedSupport()
fixedSupport.Location = ExtAPI.DataModel.GetObjectsByName("NSFixedSupportFaces")[0]

frictionlessSupport = analysis.AddFrictionlessSupport()
frictionlessSupport.Location = ExtAPI.DataModel.GetObjectsByName("NSFrictionlessSupportFaces")[0]

pressure = analysis.AddPressure()
pressure.Location = ExtAPI.DataModel.GetObjectsByName("NSInsideFaces")[0]
pressure.Magnitude.Inputs[0].DiscreteValues = [Quantity("0 [s]"), Quantity("1 [s]")]
pressure.Magnitude.Output.DiscreteValues = [Quantity("0 [Pa]"), Quantity("15 [MPa]")]

In [ ]:

# Solve model
analysis.Solve()

e_app.save(mechdat_filepath)

In [ ]:

# Add results
solution = analysis.Solution
solution.AddTotalDeformation()
solution.AddEquivalentStress()
solution.EvaluateAllResults()
e_app.save(mechdat_filepath)

# Export result values to a text file
results =  solution.GetChildren(DataModelObjectCategory.Result,True)
for result in results:
    fileName = str(result.Name)
    path = os.path.join(outputs_dir,fileName+r".txt")
    result.ExportToTextFile(True,path)


In [ ]:
e_app.close()